In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import warnings
import gc
import numpy as np
import pandas as pd
import plotly as plt
import plotly.express as px
import seaborn as sns
#import cudf
#import cupy
from numba import cuda
import pyarrow.feather as feather
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.layers.core import Activation
from keras import backend as K
from keras.datasets import mnist
from keras.datasets import boston_housing
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import load_model


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import shuffle
import sklearn

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)

In [ ]:
#Read in the data
df1 = pd.read_feather('../input/amexfeather/train_data.ftr')

In [ ]:
#Create the customer level data, taking the most recent recprd
df_train =  (df1
            .groupby('customer_ID')
            .tail(1))
df_train.S_2 = 3

In [ ]:
#Understand how many customers feature in the data
customercounts = df_train['customer_ID'].nunique()
print('The number of unique customers is: ', customercounts)

In [ ]:
#Understand how many statement the customers receive
customerstatements = df1.groupby('customer_ID')['customer_ID'].agg('count').value_counts()
px.bar(customerstatements, labels = dict(index = 'Number of months in data', value = 'Number of customers'), title = 'Number of statements in data')

In [ ]:
del df1
gc.collect()

In [ ]:
#Understand the imbalance of the target variable
imbalance = df_train.groupby('target')['customer_ID'].agg('count')
px.bar(imbalance,labels = dict(index = 'Default', value = 'Number of customers'), title = 'Class imbalance of default')

# Impute blanks and NAN in train dataset


In [ ]:
# Create feature lists
all_features = ['customer_ID','S_2','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64','B_1','B_10','B_11',
         'B_12','B_13','B_14','B_18','B_19','B_2','B_20','B_21','B_22','B_23','B_24','B_25','B_26','B_3','B_33','B_36','B_37',
         'B_4','B_40','B_5','B_6','B_7','B_8','B_9','D_112','D_113','D_115','D_118','D_119','D_121','D_122','D_128','D_129',
         'D_131','D_133','D_39','D_41','D_44','D_45','D_47','D_48','D_51','D_52','D_54','D_55','D_58','D_59','D_61','D_62',
         'D_65','D_71','D_74','D_75','D_78','D_79','D_81','D_84','P_2','P_3','P_4','R_1','R_10','R_11','R_16','R_2','R_27',
         'R_3','R_4','R_5','R_6','R_7','S_13','S_15','S_16','S_22','S_23','S_24','S_25','S_26','S_3','S_7','S_8','target']

cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64'] 

num_features = list(set(all_features) - set(cat_features) - set(['customer_ID','target']))

feature_columns = (
    num_features + cat_features + ['target'])

In [ ]:
dfnum = df_train[num_features]
dfcat = df_train[cat_features]

In [ ]:
#Replace the blanks with the median of the non blank cells by column
nona = dfnum.dropna()

for col in num_features:
    dfnum[col] = dfnum[col].fillna(nona[col].median())

In [ ]:
del nona
gc.collect()

In [ ]:
dfcat['D_64'] = dfcat['D_64'].replace([""],"unknown")

In [ ]:
for col in cat_features:
    dfcat[col] = dfcat[col].fillna(dfcat[col].value_counts().idxmax())   
    
print(dfcat.isnull().sum())

In [ ]:
imputed_train = pd.concat([dfcat, dfnum], axis=1, join='inner')
imputed_train

In [ ]:
imputed_train['target'] = df_train.target

# Read in the test data


In [ ]:
#Read in the test data and take the last record

test_df1 = pd.read_feather('/kaggle/input/amexfeather/test_data.ftr')

test_df2 =  (test_df1
            .groupby('customer_ID')
            .tail(1))

In [ ]:
testfilter = all_features.copy()
testfilter = list(set(testfilter) - set(['target']))

In [ ]:
df_test = test_df2[testfilter]


del test_df1, test_df2
gc.collect()

In [ ]:
df_test["S_2"] = df_test["S_2"].dt.month

# Process the train dataset (resample and encode categorical variables)

In [ ]:
## Importing resample from *sklearn.utils* package.
from sklearn.utils import resample

# Separate the case of default and no default
df_0 = imputed_train[imputed_train.target == 0]
df_1 = imputed_train[imputed_train.target == 1]

## Downsample the majority class
df_majority_downsampled = resample(df_0, 
                                 replace=False,     # sample with replacement
                                 n_samples=len(df_1)) 

# Combine majority class with upsampled minority class
new_df = pd.concat([df_1, df_majority_downsampled])


from sklearn.utils import shuffle
new_df = shuffle(new_df)
new_df

In [ ]:
encoded_df = pd.get_dummies( new_df, 
                                        columns = cat_features,
                                        drop_first = True )

encoded_df

In [ ]:
encoded_df.target

In [ ]:
encoded_df = encoded_df.drop(['D_64_-1'],axis=1)

In [ ]:
df_train = encoded_df

In [ ]:
y = encoded_df.target

encoded_df = encoded_df.drop("target",axis=1)

X = encoded_df

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    random_state = 4,
                                                    shuffle = True)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Build the model

In [ ]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
]

In [ ]:
model = Sequential()

model.add(Dense(113, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05)

In [ ]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=METRICS)

In [ ]:
h = model.fit(X_train,y_train,epochs = 6, batch_size = 10, validation_data=(X_test, y_test))

# Build the model with cross_val_score (not using)

In [ ]:
#def baseline_model():
    #model = Sequential()
    #model.add(Dense(113, activation='relu'))
    #model.add(Dense(30, activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
   # model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=METRICS)
   #return model

In [ ]:
#estimators = KerasClassifier(build_fn=baseline_model,epochs=10, batch_size=50)

In [ ]:
#kfold = KFold(n_splits=2, shuffle = True, random_state = 1)

#results = cross_val_score(estimators, X_train, y_train, cv=kfold)

#print(results)

In [ ]:
history_dict = h.history

In [ ]:
auc = history_dict['auc']
val_auc = history_dict['val_auc']
epochs = range(1,7)
fig = px.line(x = epochs, y = [auc,val_auc], labels = dict(auc = 'hello', val_auc = 'goodbye'))
fig.update_layout(xaxis_title="Epochs", yaxis_title="AUC", title = 'AUC by epochs')

newnames = {'wide_variable_0':'auc', 'wide_variable_1': 'val_auc'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

In [ ]:
model.summary()

# Predict the results on the test data

In [ ]:
predictions = model.predict(X_test).ravel()

In [ ]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, predictions)

In [ ]:
import plotly.express as px
px.line(x = fpr_keras, y = tpr_keras, title = 'ROC curve')

In [ ]:
del encoded_df

gc.collect()

In [ ]:
del new_df
gc.collect()

In [ ]:
del X_train
gc.collect()

In [ ]:
del dfcat
gc.collect()

In [ ]:
del dfnum
gc.collect()

In [ ]:
del imputed_train

gc.collect()

In [ ]:
del df_train
gc.collect()